Реализация решающего дерева

In [175]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import datasets
from pandas import DataFrame

boston = datasets.load_boston()
boston_frame = DataFrame(boston.data)
boston_frame.columns = boston.feature_names
boston_frame.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [176]:
data = boston.data
target = boston.target

train_data, test_data, train_target, test_target = train_test_split(data, target, test_size = 0.1)   

In [177]:
import random as rd
import numpy as np

class Node:
    def __init__(self, data):
        self.data = data
        self.children = []

    def add_child(self, x):
        self.children.append(x)
        
class DecisionTree:
    def __init__(self, criterion = 'mse', msl = 1, mss = 5):# minimal samples split = mss, minimal samples leaf = msl
        self.node = Node(0)
        self.msl = msl
        self.mss = mss
    def fit(self, x, y):
        self.node = self.split(x, y)
    def predict(self, x):
        y = [0] * len(x)
        summ = 0
        for j in x:
            #берем начальный узел
            node = self.node
            #начинаем спускаться
            while type(node) != np.int64 and type(node) != np.float64:
                ind = node.data[0]
                if j[ind] < node.data[1]:
                    node = node.children[0]
                else:
                    node = node.children[1]
            y[summ] = node
            summ += 1
        return y
    def split(self, x, y):
        second_return = np.mean(y)
        def mse(leaf):
            result = 0
            av = np.mean(leaf)
            for i in leaf:
                result += (av - i) ** 2
            return result
        def for_first_leaf(data1, data2, a, b):
            return [data1[i] for i in range(0, len(data1)) if data2[i][a] < b]
        def for_second_leaf(data1, data2, a, b):
            return [data1[i] for i in range(0, len(data1)) if data2[i][a] >= b]
        #для определения какой признак взять на данном этапе узла, т.е. тот, который даст наилучшее разбиение
        def quality(a, b):
            #разделяем по a и b
            leaf_data_1 = for_first_leaf(x, x, a, b)
            leaf_target_1 = for_first_leaf(y, x, a, b)
            leaf_data_2 = for_second_leaf(x, x, a, b)
            leaf_target_2 = for_second_leaf(y, x, a, b)
            size_1 = len(leaf_data_1)
            size_2 = len(leaf_data_2)
            if size_1 == 0 or size_2 == 0:
                return -1
            result = (size_1 * mse(leaf_target_1) + size_2 * mse(leaf_target_2)) / len(x) 
            return result
        #если в листе мало сэмплов, то выдаем ответ
        if len(x) < self.mss:
            return second_return
        size_0 = len(x[0])
        result = None
        #ищем признак, который даст ненулевое качество, т.е фича по которой можно разбить
        for a in range(0, size_0):
            array = []
            for i in x:
                array.append(i[a])
            for b in np.linspace(min(array), max(array), 100):
                res = quality(a, b)
                if res > 0:
                    result = res 
                    res_a = a
                    res_b = b
                    break
        if result == None:
            return second_return
        #проходимся по остальным и ищем те, которые лучше, найденной выше 
        for a in range(0, size_0):
            array = []
            for i in x:
                array.append(i[a])
            for b in np.linspace(min(array), max(array), 100):
                res = quality(a, b)
                if res < result and res != -1:
                    result = res
                    res_a = a
                    res_b = b
        #разделяем по res_a, res_b
        leaf_data_1 = for_first_leaf(x, x, res_a, res_b)
        leaf_target_1 = for_first_leaf(y, x, res_a, res_b)
        leaf_data_2 = for_second_leaf(x, x, res_a, res_b)        
        leaf_target_2 = for_second_leaf(y, x, res_a, res_b)
        
        size_1 = len(leaf_data_1)
        size_2 = len(leaf_data_2)
        
        if (size_1 >= self.msl and size_2 >= self.msl and len(y) >= self.mss):
            #создаем новую ноду по найденным фичам
            node = Node([res_a, res_b])
            #делим на лево
            leaf_node = self.split(leaf_data_1, leaf_target_1)
            node.add_child(leaf_node)
            #делим на право
            leaf_node = self.split(leaf_data_2, leaf_target_2)
            node.add_child(leaf_node)
            return node
        else:
            return second_return

In [178]:
my_tree = DecisionTree()
my_tree.fit(train_data, train_target)
test_predictions = my_tree.predict(test_data)
print (abs(np.array(test_target) - np.array(test_predictions))).mean()

3.29656862745
